In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
telco=pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
telco.head()

## **Data cleaning**

In [ ]:
telco.info()

**Problems in dataset**
1. In this data set cutomer id is not usable
2. Total charges dtypes is object, so it must be float64 type

In [ ]:
telco=telco.iloc[:,1:]

In [ ]:
telco.info()

In [ ]:
# pd.to_numeric(telco['TotalCharges'])
telco['TotalCharges'][488]

In [ ]:
telco=telco[telco['TotalCharges']!=' ']

In [ ]:
telco.reset_index(drop=True)

In [ ]:
telco['TotalCharges']=telco['TotalCharges'].astype('float64')

In [ ]:
telco.info()

In [ ]:
for i in telco:
  print(f'{i}:{telco[i].unique()}')

There is some No internet service feature in some columns so we have to change it to No.

In [ ]:
telco.columns

In [ ]:
for i in range(len(telco.columns)):
  if i in range(9,15):
    telco[telco.columns[i]]=telco[telco.columns[i]].replace('No internet service','No')

the multiple lines columns have no phone service which means No. So we have to change is also

In [ ]:
telco['MultipleLines']=telco['MultipleLines'].replace('No phone service','No')

In [ ]:
for i in telco:
  print(f'{i}:{telco[i].unique()}')

In [ ]:
edadata=telco.copy()

# Data Preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
z=telco.select_dtypes(include='object')
for i in z:
  print(i)
  z[i]=le.fit_transform(z[i])

In [ ]:
for i in z:
  print(f'{i}:{z[i].unique()}')

In [ ]:
for i in telco:
  for j in z:
    if i==j:
      telco[i]=z[j]

In [ ]:
telco

now dataset have all numeric values in it but InternetService, contract and paymentmethod is in 3 or 4 decision choices so we have to change it according to one hot encoding

In [ ]:
telco=pd.get_dummies(data=telco,columns=['Contract','PaymentMethod','InternetService'])
telco

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
telco[['tenure','MonthlyCharges','TotalCharges']]=scaler.fit_transform(telco[['tenure','MonthlyCharges','TotalCharges']])
telco

# EDA of DATASET

In [ ]:
edadata

lets see the data 

In [ ]:
import seaborn as sns

In [ ]:

sns.factorplot(data=edadata,x='Churn',y='tenure',hue='gender')

*OBSERVATION:Tenure is inversely porportional to Churn, as per the graph those customer whose tenure is large , they do not left the company and viceversa.*

In [ ]:
f,axes=plt.subplots(1,2,figsize=(16,9))
sns.barplot(ax=axes[0],x='tenure',y='Churn',hue='Contract',data=edadata)
sns.boxplot(ax=axes[1],x='tenure',y='Churn',hue='Contract',data=edadata)

*Observation: As per given graph, if customer whose tenure is low haveing month-to-month contract and tenure with middle range have one year contract and higher tenure peope have two year contract*

In [ ]:
edadata.columns


In [ ]:

sns.factorplot(x='tenure',y='TotalCharges',hue='Churn',data=edadata)

*Observation: there are more customer who churn as there tenure and total charges are increasing *

# Train and Test split 

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(telco.drop(columns='Churn'),telco.Churn,test_size=0.3,random_state=7)

# Prediction 

In [ ]:
import tensorflow as tf
from tensorflow import keras
model=keras.Sequential([
                        keras.layers.Dense(20,input_shape=(len(X_train.columns),),activation='relu'),
                        keras.layers.Dense(15,activation='relu'),
                        keras.layers.Dense(10,activation='relu'),
                        keras.layers.Dense(5,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train,y_train,epochs=50)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
l=model.predict(X_test)

In [ ]:
l

In [ ]:
li=[]
for i in range(len(l)):
  if l[i][0]<0.5:
    li.append(0)
  else:
    li.append(1)

In [ ]:
li[:6]

In [ ]:
y_test[:6]

# Accuracy and Classification

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test,li))

In [ ]:
cm=tf.math.confusion_matrix(labels=y_test,predictions=li)

In [ ]:
sns.heatmap(cm,annot=True,fmt='d')
plt.xlabel('predicted')
plt.ylabel('Truth')
